Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect

# Data Preprocessing & Cleaning for NMT

This notebook contains a tutorial of data processing and cleaning for NMT (Neural Machine Translation) to train translation models with the [NeMo framework](https://github.com/NVIDIA/NeMo).

A pre-requisite to train supervised neural machine translation systems is the availability of *parallel corpora* of reasonable quality.

A parallel corpus is a collection of sentences or documents that are translations of each other in 2 or more languages.

For example,

| English                                                                            | Russian |
| :-: | :-: |
| To date, a total of 43 participants from 15 countries have completed the training. | К настоящему времени подготовку прошли в общей сложности 43 участника из 15 стран . |
| M-Sport Bentley writes a new piece of Bentley history at Silverstone | M-Sport Bentley открывает новую страницу в истории Bentley в Сильверстоуне |
| Information in the application was not true. | Информация в заявлении не была достоверна. |

This notebook will cover the following data pre-processing and data cleaning techniques for such corpora.

## The importance of data cleaning

The presence of noise in the training dataset can adversely affect model quality (https://arxiv.org/abs/1805.12282). Webcrawled and automatically aligned data sources in particular, such as [Paracrawl](https://paracrawl.eu/), [WikiMatrix](https://arxiv.org/abs/1907.05791), [CC-Aligned](https://arxiv.org/abs/1911.06154) and [CC-Matrix](https://arxiv.org/abs/1911.04944) can be extremely noisy.

## Cleaning
1. Downloading and filtering publicly available datasets based on confidence thresholds (if available). For example, [WikiMatrix](https://arxiv.org/abs/1907.05791) filtering based on [LASER](https://arxiv.org/abs/1812.10464) confidence scores.
2. Language ID filtering using a pre-trained [fastText classifier](https://fasttext.cc/docs/en/language-identification.html). This step will remove all sentences from the parallel corpus that our classifier predicts as not being in the appropriate language (ex: sentences in the English column that aren't in English or sentences in Russian column that aren't in Russian).
3. Length and Length-ratio filtering. This steps removes all sentences that are 1) too long 2) too short or 3) have a ratio between their lengths greater than a certain factor (this typically removes partial translations).
4. [Bicleaner](https://github.com/bitextor/bicleaner) classifier-based cleaning. Bicleaner identifies noisy parallel sentences using a classifier that leverages multiple features such as n-gram language model likelihood scores, word alignment scores and other heuristics.

## Pre-processing
5. [Moses Punctuation Normalization](https://github.com/moses-smt/mosesdecoder/blob/master/scripts/tokenizer/normalize-punctuation.perl). This step standardizes punctuation. For example the less common way to write apostrophes Tiffany`s will be standardized to Tiffany's.
6. Unicode standardization. There exist some unicode characters that aren't punctuation that need to be standardized for example, this step normalizes the number ４ to 4.
7. [Moses Tokenization](https://github.com/moses-smt/mosesdecoder/blob/master/scripts/tokenizer/tokenizer.perl) or text segmentation for Chinese/Japanese with [Jieba](https://github.com/fxsjy/jieba) and [mecab](https://github.com/taku910/mecab). For languages like Chinese and Japanese that do not have explicit word segmentation markers (like spaces), we use these tools to introduce spaces into the text that will let us split the string into words. For other languages, we use Moses to separate punctuation markers from words so that they become separate tokens.
8. Deduplication - This step removes duplicate translation pairs from the corpus.
9. Shuffling - This step shuffles the order of occurrence of translation pairs.

## Tarred Datasets for Large Corpora
10. Large datasets with over 50M sentence pairs when batched and pickled can be up to 60GB in size. Loading them entirely into CPU memory when using say 8 or 16 workers with DistributedDataParallel training uses 480-960GB of RAM which is often impractical and inefficient. Instead, we use [Webdataset](https://github.com/webdataset/webdataset) to allow training while keeping datasets on disk and let webddataset handle pre-loading and fetching of data into CPU RAM.


## Disclaimer

The data cleaning techniques used in this notebook are only meant to be loose guidelines and are not guaranteed to produced clean parallel corpora at the end of it. Not all of these steps are a necessity for every dataset, 

![NMT Data Pipeline](images/nmt_data_pipeline.png)

# Downloading Publicly Available Data

## WikiMatrix (https://arxiv.org/abs/1907.05791)

In [ ]:
!mkdir -p data
print('Downloading data ...')
!wget https://dl.fbaipublicfiles.com/laser/WikiMatrix/v1/WikiMatrix.en-ru.tsv.gz -O data/WikiMatrix.en-ru.tsv.gz
print('---------------------')
print('Unzipping file ...')
!gunzip -k -f data/WikiMatrix.en-ru.tsv.gz
print('---------------------')
print('Peek into the file')
!head -10 data/WikiMatrix.en-ru.tsv
print('---------------------')
print('File length ...')
!wc -l data/WikiMatrix.en-ru.tsv
print('---------------------')

## Filter Based on LASER Confidence

LASER (https://arxiv.org/abs/1812.10464) is a multi-lingual neural sentence embedding model that is often used for cross-lingual sentence/document retrieval. Similarities in the embedding space are often used as proxies for cross-lingual similarities.

In [ ]:
from tqdm import tqdm
import numpy as np

def num_lines_in_file(fname):
    """
    Returns the number of lines in a file.
    """
    with open(fname, 'r') as f:
        for i, _ in enumerate(f):
            pass
    return i + 1

def filter_tsv_with_conf(
    input_file, output_file_lang_1, output_file_lang_2,
    confidence_threshold=None, confidence_column=None
):
    """
    Filters a tsv file that has confidence scores associated with each parallel example.

    For example:

    1.23 \t This is a sentence in lang1 \t This is a sentence in lang2
    """
    print()
    print('====================================')
    print('======= TSV Conf Filtering =========')
    print('====================================')
    print()
    num_lines = num_lines_in_file(input_file)
    scores = []
    num_output_lines = 0
    lang_1_col = 0
    lang_2_col = 1
    with open(input_file, 'r') as f, \
        open(output_file_lang_1, 'w') as f_out_1, \
        open(output_file_lang_2, 'w') as f_out_2:
        for line in tqdm(f, total=num_lines, desc=f"Filtering file by confidence {confidence_threshold}"):
            if line.strip() == '':
                continue
            line = line.strip().split('\t')
            if len(line) < 2:
                continue
            if confidence_threshold is not None and float(line[confidence_column]) < confidence_threshold:
                continue
            else:
                if confidence_threshold is not None:
                    scores.append(float(line[confidence_column]))
                    if confidence_column == 0:
                        lang_1_col, lang_2_col = 1, 2
                    elif confidence_column == 2:
                        lang_1_col, lang_2_col = 0, 1
                    elif confidence_column == 1:
                        lang_1_col, lang_2_col = 0, 2
                    else:
                        raise ValueError(f"Invalid Column for confidence {confidence_column}")
                f_out_1.write(line[lang_1_col] + '\n')
                f_out_2.write(line[lang_2_col] + '\n')
                num_output_lines += 1

    if confidence_threshold is not None:
        print(f'Confidence score average  : {np.mean(scores)}')
        print(f'Confidence score variance : {np.var(scores)}')
        print(f'Kept {num_output_lines} out of {num_lines} after conversion ({(num_output_lines / num_lines) * 100}%)')
        print('====================================')

filter_tsv_with_conf(
    'data/WikiMatrix.en-ru.tsv',
    'data/WikiMatrix.en-ru.en', 
    'data/WikiMatrix.en-ru.ru',
    confidence_threshold=1.04, confidence_column=0
)

## Language ID filtering with fastText

Noisy parallel corpora often contain sentences that are not in the intended language. A classifier that determines the language in which a sentence is written can be used to filter out sentences that aren't in the appropriate language.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -O data/lid.176.bin
print()
print('====================================')
print('====== Language ID Filtering =======')
print('====================================')
print()


!wget https://raw.github.com/NVIDIA/NeMo/main/scripts/neural_machine_translation/filter_langs_nmt.py \
    -O filter_langs_nmt.py

!python filter_langs_nmt.py \
    --input-src data/WikiMatrix.en-ru.en  \
    --input-tgt data/WikiMatrix.en-ru.ru \
    --output-src data/WikiMatrix.en-ru.langidfilter.en  \
    --output-tgt data/WikiMatrix.en-ru.langidfilter.ru  \
    --source-lang en \
    --target-lang ru \
    --removed-src data/WikiMatrix.en-ru.langidfilter.removed.en  \
    --removed-tgt data/WikiMatrix.en-ru.langidfilter.removed.ru  \
    --fasttext-model data/lid.176.bin

print()
print('-----------------------------------------')
print('Number of removed sentences:')
print('-----------------------------------------')
print()
!wc -l data/WikiMatrix.en-ru.langidfilter.removed.ru

print()
print('-----------------------------------------')
print('Examples of removed sentences')
print('-----------------------------------------')
print()

!paste -d "\t" \
    data/WikiMatrix.en-ru.langidfilter.removed.en \
    data/WikiMatrix.en-ru.langidfilter.removed.ru \
    | head -10
print('-----------------------------------------')

## Length and Ratio Filtering

This step filters out sentences based on their lengths and the ratio between source and target lengths. If (a) src_len / tgt_len or tgt_len / src_len exceed 1.3 or (b) source or target sequence lengths are less than 1 or greater than 250, the sentence pair will be removed.

In [ ]:
!git clone https://github.com/moses-smt/mosesdecoder data/mosesdecoder
!cd data/mosesdecoder && git checkout RELEASE-4.0 && cd ../..
!perl data/mosesdecoder/scripts/training/clean-corpus-n.perl -ratio 1.3 \
    data/WikiMatrix.en-ru.langidfilter \
    en ru \
    data/WikiMatrix.en-ru.langidfilter.lengthratio \
    1 250

<font color='red'>THE FOLLOWING CELLS REQUIRE THE INSTALLATION OF BICLEANER, WHICH REQUIRES COMPILING PACKAGES FROM SOURCE AND IS TRICKY TO GET WORKING INSIDE THIS CONTAINER. PLEASE INSTALL BICLEANER FROM THE REPOSITORY - https://github.com/bitextor/bicleaner OR FOLLOW INSTRUCTIONS BELOW. CELLS FOLLOWING THIS WILL NOT RUN IF BICLEANER IS NOT INSTALLED.</font>

You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

## Install dependencies

!pip install wget
!apt-get install libboost-all-dev
!apt-get install gawk

## Install NeMo

BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

!pip uninstall -y sacrebleu
!pip install sacrebleu[ja]
!pip install xxhash

## Install kenlm with 7-gram support
!mkdir -p data
!rm -rf data/kenlm
!git clone https://github.com/kpu/kenlm data/kenlm
!cd data/kenlm \
    && pip install . --install-option="--max_order 7" \
    && mkdir -p build \
    && cd build \
    && cmake .. -DKENLM_MAX_ORDER=7 -DCMAKE_INSTALL_PREFIX:PATH=../../kenlm_install \
    && make -j all install && cd ../../kenlm_install \
    && export PATH=$PATH:$PWD

# Install bicleaner

!pip install bicleaner

In [ ]:
try:
    import bicleaner
except ImportError:
    raise ImportError(f"You need to install Bicleaner to proceed. Could not import the bicleaner package.")

## Bicleaner Filtering

Bicleaner (https://aclanthology.org/W18-6488/ and https://aclanthology.org/2020.eamt-1.31/) is a tool to identify noisy parallel sentences in translation corpora. It applies 3 different filtering steps:

1. Pre-filtering based on 37 rules.
2. Language model fluency scores based on n-gram language models trained with kenlm.
3. Random forest classifier that uses all examples filtered out in steps 1 & 2 as "negative" examples.

In [ ]:
print('Downloading En-Ru Bicleaner models.')
!git clone https://github.com/bitextor/bicleaner data/bicleaner
!cd data/bicleaner && git checkout bicleaner-0.15 && cd ../..
!data/bicleaner/utils/download-pack.sh en ru

print('Generating Bicleaner scores ...')
!gawk '{{print "-\t-"}}' \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.en | \
    paste -d "\t" - data/WikiMatrix.en-ru.langidfilter.lengthratio.en \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.ru | \
    bicleaner-classify - - en-ru/en-ru.yaml \
    > data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.scores

In [ ]:
print('Score file ...')
!head -10 data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.scores

print()
print('-----------------------------------------')
print('Filtering based on Bicleaner scores > 0.6 ...')
print('-----------------------------------------')
print()

print('Filtering out English ...')
!gawk -F "\t" '{if ($5>0.6) {print $3}}' \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.scores > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.en

print('Filtering out Russian ...')
!gawk -F "\t" '{if ($5>0.6) {print $4}}' \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.scores > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.ru

!paste -d "\t" \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.en \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.ru \
    | head -10

## Normalize Punctuation

Punctuation can vary across languages and even between ascii and unicode variants of the same punctuation marker. For example, across languages. For example, in German, quotes are often written as „ and “ while in English we typically just use ". This step normalizes such punctuation differences to use the same character everywhere.

We use [moses](https://github.com/moses-smt/mosesdecoder) or [sacremoses](https://github.com/alvations/sacremoses) to normalize punctuation. The moses implementation is in perl while sacremoses is in python with a CLI interface. The perl implementation is buffered and works better for large corpora that may not fit into CPU memory all at once while sacremoses is unbuffered and multi-processed.

### Sacremoses

In [ ]:
print('Normalizing English ...')
!sacremoses -j 4 normalize \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.en > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.en

print('Normalizing Russian ...')
!sacremoses -j 4 normalize \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.ru > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.ru


## Moses

Punctuation can vary across languages and even between ascii and unicode variants of the same punctuation marker. For example, across languages. For example, in German, quotes are often written as „ and “ while in English we typically just use ". This step normalizes such punctuation differences to use the same character everywhere.

We use [moses](https://github.com/moses-smt/mosesdecoder) or [sacremoses](https://github.com/alvations/sacremoses) to normalize punctuation. The moses implementation is in perl while sacremoses is in python with a CLI interface. The perl implementation is buffered and works better for large corpora that may not fit into CPU memory all at once while sacremoses is unbuffered and multi-processed.

In [ ]:
print('Normalizing English ...')
!perl data/mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l en \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.en > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.en

print('Normalizing Russian ...')
!perl data/mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l ru \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.ru > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.ru


## Tokenize

Tokenization splits a string into a sequence of tokens. A naive way of doing this would be to simply split the string on spaces (for languages where this is possible). This however, will result in punctuation being "attached" to the neighboring word when tokenizing. For example, 

"This is a sentence." will be tokenized as ["This, is, a, sentence."].

However, we'd typically like punctuation to be separate tokens for example,

"This is a sentence." will be tokenized my moses or sacremoses as [", This, is, a, sentence, ., "].

### Sacremoses

In [ ]:
print('Tokenizing English ...')
!sacremoses -j 4 -l en tokenize -x \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.en > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.tok.en

print('Tokenizing Russian ...')
!sacremoses -j 4 -l ru tokenize -x \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.ru > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.sacremoses.norm.tok.ru


### Moses

In [ ]:
print('Tokenizing English ...')
!perl data/mosesdecoder/scripts/tokenizer/tokenizer.perl -l en -no-escape -threads 4 \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.en > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.en

print('Tokenizing Russian ...')
!perl data/mosesdecoder/scripts/tokenizer/tokenizer.perl -l ru -no-escape -threads 4 \
    < data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.ru > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.ru


In [ ]:
print()
print('-----------------------------------------')
print('Tokenized Russian Sentences ...')
print('-----------------------------------------')
print()

!head -10 data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.ru

print()
print('-----------------------------------------')
print('Tokenized English Sentences ...')
print('-----------------------------------------')
print()

!head -10 data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.en

## Segmenting Chinese and Japanese

### Jieba segmentation for Chinese

In [ ]:
import jieba

!wget https://dl.fbaipublicfiles.com/laser/WikiMatrix/v1/WikiMatrix.en-zh.tsv.gz -O data/WikiMatrix.en-zh.tsv.gz
!gunzip -k -f data/WikiMatrix.en-zh.tsv.gz

print()
print('-----------------------------------------')
print('Chinese text before segmentation ...')
print('-----------------------------------------')
print()

!awk -F "\t" '{print $3}' data/WikiMatrix.en-zh.tsv | head -10
print()
print('-----------------------------------------')
print('Segmenting Chinese text ...')
print('-----------------------------------------')
print()

zh_lines = []
with open('data/WikiMatrix.en-zh.tsv', 'r') as f:
    for idx, line in enumerate(f):
        line = line.strip().split('\t')[2]
        zh_lines.append(' '.join(jieba.cut(line)))
        if idx == 100:
            break
print()
print('-----------------------------------------')
print('Chinese text after segmentation ...')
print('\n'.join(zh_lines[:10]))
print('-----------------------------------------')
print()

In [ ]:
import MeCab
import ipadic

!wget https://dl.fbaipublicfiles.com/laser/WikiMatrix/v1/WikiMatrix.en-ja.tsv.gz -O data/WikiMatrix.en-ja.tsv.gz
!gunzip -k -f data/WikiMatrix.en-ja.tsv.gz

print()
print('-----------------------------------------')
print('Japanese text before segmentation ...')
print('-----------------------------------------')
print()

!awk -F "\t" '{print $3}' data/WikiMatrix.en-ja.tsv | head -10

print()
print('-----------------------------------------')
print('Segmenting Japanese text ...')
print('-----------------------------------------')
print()

mecab_tokenizer = MeCab.Tagger(ipadic.MECAB_ARGS + " -Owakati")

ja_lines = []
with open('data/WikiMatrix.en-ja.tsv', 'r') as f:
    for idx, line in enumerate(f):
        line = line.strip().split('\t')[2]
        ja_lines.append(mecab_tokenizer.parse(line))
        if idx == 100:
            break
print()
print('-----------------------------------------')
print('Japanese text after segmentation ...')
print('\n'.join(ja_lines[:10]))
print('-----------------------------------------')
print()

## Deduplicate

In [ ]:
import xxhash

def dedup_file(input_file_lang_1, input_file_lang_2, output_file_lang_1, output_file_lang_2):
    print()
    print('====================================')
    print('========== De-duplicate ============')
    print('====================================')
    print()
    num_lines = num_lines_in_file(input_file_lang_1)
    hashes = set()
    num_output_lines = 0
    with open(input_file_lang_1, 'r') as f_lang1, \
        open(input_file_lang_2, 'r')  as f_lang2, \
        open(output_file_lang_1, 'w') as f_out_lang1, \
        open(output_file_lang_2, 'w') as f_out_lang2:
        for line_1, line_2 in tqdm(zip(f_lang1, f_lang2), total=num_lines, desc=f"Deduplicating files"):
            parallel_hash = xxhash.xxh64((line_1.strip() + '\t' + line_2.strip()).encode('utf-8')).hexdigest()
            if parallel_hash not in hashes:
                hashes.add(parallel_hash)
                f_out_lang1.write(line_1.strip() + '\n')
                f_out_lang2.write(line_2.strip() + '\n')
                num_output_lines += 1

    print(f"Kept {num_output_lines} out of {num_lines} after deduplication")

dedup_file(
    'data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.en',
    'data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.ru',
    'data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.en',
    'data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.ru'
)

## Shuffle

In [ ]:
!shuf --random-source=data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.en \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.en > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.en

!shuf --random-source=data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.en \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.ru > \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.ru

!paste -d "\t" \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.en \
    data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.ru \
    | head -10

In [ ]:
!rm -rf data/tarred_dataset_en_ru_8k_tokens

## Tarred Dataset Creation

In [ ]:
!wget https://raw.github.com/NVIDIA/NeMo/main/examples/nlp/machine_translation/create_tarred_parallel_dataset.py \
    -O create_tarred_parallel_dataset.py

!python create_tarred_parallel_dataset.py \
    --src_fname data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.en \
    --tgt_fname data/WikiMatrix.en-ru.langidfilter.lengthratio.bicleaner.60.moses.norm.tok.dedup.shuf.ru \
    --out_dir data/tarred_dataset_en_ru_8k_tokens \
    --clean \
    --encoder_tokenizer_name yttm \
    --encoder_tokenizer_vocab_size 32000 \
    --encoder_tokenizer_coverage 0.999 \
    --encoder_tokenizer_bpe_dropout 0.1 \
    --decoder_tokenizer_name yttm \
    --decoder_tokenizer_vocab_size 32000 \
    --decoder_tokenizer_coverage 0.999 \
    --decoder_tokenizer_bpe_dropout 0.1 \
    --max_seq_length 512 \
    --min_seq_length 1 \
    --tokens_in_batch 8000 \
    --lines_per_dataset_fragment 100000 \
    --num_batches_per_tarfile 20


In [ ]:
!ls data/tarred_dataset_en_ru_8k_tokens

In [ ]:
!cat data/tarred_dataset_en_ru_8k_tokens/metadata.tokens.8000.json